In [ ]:
import cv2
import os
os.chdir('D:\DL Projects\Age and Gender Detection\models')

In [ ]:
#Pre-processing

def detectFace(net,frame,confidence_threshold=0.7):
    frameOpencvDNN=frame.copy()
    print(frameOpencvDNN.shape)
    frameHeight=frameOpencvDNN.shape[0]
    frameWidth=frameOpencvDNN.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDNN,1.0,(227,227),[124.96,115.97,106.13],swapRB=True,crop=False) #preprocessing whole frame  #RGB mean #process it with blob
    net.setInput(blob)
    detections=net.forward() # to get output process on input and give it to next frame
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>confidence_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)       # declaring dimensions for facebox acc. to height & width
            y1=int(detections[0,0,i,4]*frameHeight)      # 4D arrays - with 2nd index as i which compares to threshold 
            x2=int(detections[0,0,i,5]*frameWidth)       #indexing 3,4,5,6
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDNN,(x1,y1),(x2,y2),(0,255,0),int(round(frameHeight/150)),8)
    return frameOpencvDNN,faceBoxes

#calling the pretrained models
    
faceProto='opencv_face_detector.pbtxt'
faceModel='opencv_face_detector_uint8.pb'
ageProto='age_deploy.prototxt'
ageModel='age_net.caffemodel'
genderProto='gender_deploy.prototxt'
genderModel='gender_net.caffemodel'

#facebox labelling

genderList=['Male','Female']
ageList=['(0-2)','(4-6)','(8-12)','(15-20)','(25-32)','(38-43)','(48-53)','(60-100)']

#Loading in the neural network

faceNet=cv2.dnn.readNet(faceModel,faceProto)
ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)

#live video capturing

video=cv2.VideoCapture(0)    # 0 - inbuilt webcam it will take & 1 - External camera
padding=20
while cv2.waitKey(1)<0:     # waitkey - video is running and capturing don't close
    hasFrame,frame=video.read()   
    if not hasFrame:              # if not frame is there
        cv2.waitKey()
        break
        
    resultImg,faceBoxes=detectFace(faceNet,frame)
    
    if not faceBoxes:            # if not any facebox is detecting
        print("No face detected")
    
    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding):min(faceBox[2]+padding, frame.shape[1]-1)] #preprocess for face for more accuracy
        blob=cv2.dnn.blobFromImage(face,1.0,(227,227),[124.96,115.97,106.13],swapRB=True,crop=False)
        genderNet.setInput(blob)
        genderPreds=genderNet.forward()
        gender=genderList[genderPreds[0].argmax()]
        
        ageNet.setInput(blob)
        agePreds=ageNet.forward()
        age=ageList[agePreds[0].argmax()]
        cv2.putText(resultImg,f'{gender},{age}',(faceBox[0],faceBox[1]-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2,cv2.LINE_AA)
        cv2.imshow("Detecting age and Gender",resultImg)
        
        
        if cv2.waitKey(33) & 0xFF == ord('q'):
            break
            
cv2.destroyAllWindows()
        